# Week 03 Assignment: Huffman Encoding


## Technical notes


In [ ]:
##12345678901234567890123456789012345678901234567890123456789012345678901234567890

from Node import Node

_LETTERS = 26 # Number of letters in the English alphabet
_ASCII_A = 65 # ASCII value of 'A'
_SPACE_INDEX = 26 # Index of space in the encoding table
_SPACE = " " # Space character
_LEFT = "0" # Left branch in the Huffman tree
_RIGHT = "1" # Right branch in the Huffman tree
_BITS_PER_BYTE = 8 # Number of bits per byte


def filter_uppercase_and_spaces(input_string: str) -> str:
    """
    Filters the input string to retain only uppercase letters and spaces.
    """
    return "".join(
        char for char in input_string.upper() if char.isalpha() or char == " "
    )


def count_frequencies(input_string: str) -> list[int]:
    """
    Counts the frequency of each uppercase letter in the input string.
    Returns a list of 26 integers, where index 0-25 correspond to 'A'-'Z'.
    You can assume the input string contains only uppercase letters and spaces.
    And that spaces are the most frequent character, so really we dont need
    to count them.
    """
    cleaned_string = filter_uppercase_and_spaces(input_string)
    frequencies = [0] * _LETTERS  # One slot for each letter A-Z
    for char in cleaned_string:
        # char guaranteed to be alpha here, don't process spaces
        if char != " ":
            index = ord(char) - _ASCII_A
            frequencies[index] += 1
    return frequencies


def initialize_forest(frequencies: list[int]) -> list[Node]:
    """
    Initializes a forest (list) of Node objects for each character with a non-zero frequency.
    """
    forest = []
    highest = 0
    for i in range(len(frequencies)):
        if frequencies[i] > 0:
            forest.append(Node(frequencies[i], chr(i + _ASCII_A)))
            highest = frequencies[i] if frequencies[i] > highest else highest
    # Add space character with frequency equal to total of all other characters.
    # This ensures space is the most frequent character.
    forest.append(Node(1 + highest, _SPACE))
    return forest


def get_smallest_node(forest: list[Node]) -> Node:
    """
    Finds and removes the Node with the smallest frequency from the forest.
    Returns that Node.
    """
    smallest_index = 0
    for i in range(1, len(forest)):
        if forest[i].get_frequency() < forest[smallest_index].get_frequency():
            smallest_index = i
    return forest.pop(smallest_index)


def build_huffman_tree(frequencies: list[int]) -> Node:
    """
    Builds the Huffman tree from the list of frequencies and returns the root Node.
    """
    forest = initialize_forest(frequencies)
    while len(forest) > 1:
        left = get_smallest_node(forest)
        right = get_smallest_node(forest)
        merged = Node(left.get_frequency() + right.get_frequency())
        merged.set_left(left)
        merged.set_right(right)
        forest.append(merged)
    return forest[0]


def build_encoding_table(huffman_tree_root: Node) -> list[str]:
    """
    Builds the encoding table from the Huffman tree.
    Returns a list of 27 strings, where index 0-25 correspond to 'A'-'Z'
    and index 26 corresponds to space.
    Each string is the binary encoding for that character.
    """
    encoding_table = [""] * (_LETTERS + 1)  # 26 letters + 1 space
    # Iterative DFS using a stack initially containing the root node
    stack = [(huffman_tree_root, "")]
    while stack:
        # While the stack is not empty, pop a node and its path
        node, path = stack.pop()
        # If the node is a leaf, record the path in the encoding table, else
        # push the children onto the stack with updated paths
        if node.get_symbol() is not None:
            # Node is a leaf, get its character and store the path. Remember
            # that SPACE requires special handling.
            char = node.get_symbol()
            if char == _SPACE:
                # Space is stored as the 27th element of the encoding table
                index = _SPACE_INDEX
            else:
                # The rest of the characters are stored in the rest of the
                # encoding table based on their sequence in the alphabet.
                index = ord(char) - _ASCII_A
            encoding_table[index] = path
        else:
            # The node is not a leaf node, so push its children to the stack
            # updating the path with the left/right information and repeat.
            if node.get_right() is not None:
                stack.append((node.get_right(), path + _RIGHT))
            if node.get_left() is not None:
                stack.append((node.get_left(), path + _LEFT))
    return encoding_table


def encode(input_string: str, encoding_table: list[str]) -> str:
    """
    Encodes the input string using the provided encoding table. Remember
    that the encoding table has 27 entries, one for each letter A-Z and
    one for space. Space is at the last index (26).
    """
    # 
    encoded_string = ""
    # Parse the input string character by character, looking up each
    # character in the encoding table and appending the corresponding
    # binary string to the encoded string.
    for char in input_string:
        if char == _SPACE:
            # Remember that space is the last entry in the encoding table
            index = _SPACE_INDEX
        else:
            # The rest of the characters are stored in the rest of the
            # encoding table based on their sequence in the alphabet.
            index = ord(char) - _ASCII_A
        encoded_string += encoding_table[index]
    return encoded_string


def decode(encoded_string: str, huffman_root: Node) -> str:
    """
    Decodes the encoded string using the Huffman table as a key.
    """
    decoded_message = ""
    # Begin at the root of the Huffman tree and traverse it according to
    # the bits in the encoded string. When you reach a leaf node, append
    # the corresponding character to the decoded message and return to
    # the root of the tree to continue decoding.
    root: Node = huffman_root
    for bit in encoded_string:
        if bit == _LEFT:
            # Traverse left
            root = root.get_left()
        else:
            # Traverse right
            root = root.get_right()
        if root.get_symbol() is not None:
            # We've reached a leaf node, so append the character to the
            # decoded message and return to the root of the tree.
            decoded_message += root.get_symbol()
            root = huffman_root
    return decoded_message


if __name__ == "__main__":
    test_string = "Now is the winter of our discontent made glorious summer by this sun of York and all the clouds that lour'd upon our house in the deep bosom of the ocean buried".upper()
    frequencies = count_frequencies(test_string)
    print(f"All frequencies except space: {frequencies}")
    huffman_tree_root = build_huffman_tree(frequencies)
    encoding_table = build_encoding_table(huffman_tree_root)
    encoded_string = encode(test_string, encoding_table)
    decoded_string = decode(encoded_string, huffman_tree_root)
    print(f"Encoding table: {encoding_table}")
    print(f"Original: {test_string}")
    print(f"Encoded: {encoded_string}")
    print(f"Length of encoded string: {len(encoded_string)} bits")
    print(f"Length of original string: {len(test_string)*_BITS_PER_BYTE} bits")
    print(f"Decoded: {decoded_string}")

All frequencies except space: [5, 3, 3, 7, 13, 3, 1, 7, 7, 0, 1, 5, 4, 9, 17, 2, 0, 8, 9, 10, 10, 0, 2, 0, 2, 0]
Encoding table: ['11010', '111100', '111101', '11111', '1110', '00100', '0110110', '0000', '0001', '', '0110111', '11011', '00101', '0111', '010', '011000', '', '0011', '1010', '1011', '1100', '', '011001', '', '011010', '', '100']
Original: NOW IS THE WINTER OF OUR DISCONTENT MADE GLORIOUS SUMMER BY THIS SUN OF YORK AND ALL THE CLOUDS THAT LOUR'D UPON OUR HOUSE IN THE DEEP BOSOM OF THE OCEAN BURIED
Encoded: 0111010011001100000110101001011000011101000110010001011110111110001110001000100100010110000111001111100011010111101010011110111110011110111000010111010111111110100011011011011010001100010101100101010010101100001010010111100011100111100011010100101100000001101010010101100011110001000100100011010010001101101111001101001111111110011010110111101110010110000111010011110111011010110011111101010010110000110101011100110110101100001111110011111100110001100001001111000101100001110